### Imports


In [1]:
import os
import json
import datetime
from envyaml import EnvYAML
import pandas as pd
from tsfresh import extract_features
import numpy as np


### Accessing datasets


In [2]:
def read_csv(path):
    return pd.read_csv(path)


def to_csv(df, path):
    df.to_csv(path, index=False)


E4 dataset directories


In [3]:
VAR_ENV = EnvYAML("../../../env.yaml")
path_list = [os.path.join("../../..", VAR_ENV["dataset.path"], VAR_ENV["dataset.version"],
                          f"P{i:02d}", "E4") for i in range(1, VAR_ENV["dataset.participants"] + 1)]

### Dataframes


Categorical encoding for activities

Sit - 0</br>
Stand - 1</br>
Cycle(Speed 1) - 2</br>
Cycle(Speed 2) - 3</br>
Run(Speed 1) - 4</br>
Run(Speed 2) -5</br>

In [16]:
study_information = pd.read_csv(os.path.join("../../..", VAR_ENV["dataset.path"], VAR_ENV["dataset.version"],"Study_Information.csv"))
activities = ["Start_Sit", "Start_Stand", "Start_Cycle1", "Start_Cycle2", "Start_Run1", "Start_Run2"]
for path in path_list:
    df = pd.read_csv(os.path.join(path, "modified_with_tsfresh_ACC.csv"))
    # df['activity'] = np.nan
    activity = []
    # add user id
    userIds = [path_list.index(path)+1 for i in range(len(df))]
    df.insert(0, "user_id", userIds, allow_duplicates=True)

    current_activity_index = 0
    for i in range(len(df)):
        if (current_activity_index == 5):
            # df.loc[i,'activity'] = 5
            activity.append(5)
            continue
        row_end_time = datetime.datetime.fromisoformat(df.end_timestamp[i].split('.')[0])
        next_activity_start_time = datetime.datetime.fromisoformat(study_information[activities[current_activity_index+1]][path_list.index(path)])
        if (row_end_time < next_activity_start_time):
            # df.loc[i,'activity'] = current_activity_index
            activity.append(current_activity_index)
        else:
            current_activity_index += 1
            # df.loc[i,'activity'] = current_activity_index
            activity.append(current_activity_index)
    
    df.insert(1, "activity", activity, allow_duplicates=True)
    
    # saving
    to_csv(df, os.path.join("../../..",
           VAR_ENV["dataset.path"], VAR_ENV["dataset.version"], f"P{path_list.index(path)+1:02d}", "E4", "modified_with_activity_ACC.csv"))
